# Initial Analysis

#### Imports

In [2]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


### Retrieve as Variable

In [ ]:
csv_file_path = 'option_data.csv'
spydata = pd.read_csv(csv_file_path)

In [ ]:
display(spydata)

Order by the appropriate date, then expiry date, then strike price within the date spread.

In [ ]:
spydata_sorted = spydata.sort_values(by=['date', 'exdate', 'cp_flag', 'strike_price'])
#spydata_cleaned = spydata_sorted.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
#spydata_final = spydata_cleaned.reset_index(drop=True)
spydata_final = spydata_sorted.reset_index(drop=True)

In [ ]:
display(spydata_final)

In [ ]:
spydata_final.to_csv('sorteddata.csv', index=False)

Rather than download again, simply reload.

In [ ]:
spy_optiondata = pd.read_csv('sorteddata.csv')

In [ ]:
display(spy_optiondata)

Now we'll load the ticker price data, directly from local.

In [ ]:
spy_tickerdata = pd.read_csv('spy_tickerdata.csv')

In [ ]:
display(spy_tickerdata)

## Adjustments

Make some adjustments and attach to our data table for options.

In [ ]:
spy_tickerdata['date'] = pd.to_datetime(spy_tickerdata['date'])
start_date = '2018-01-01'
end_date = '2023-02-28'
spy_tickerdata_filtered = spy_tickerdata[(spy_tickerdata['date'] >= start_date) & (spy_tickerdata['date'] <= end_date)]

spy_tickerdata_sorted = spy_tickerdata_filtered.sort_values(by='date')
spy_tickerdata_reduced = spy_tickerdata_sorted[['date', 'adj_open', 'adj_high', 'adj_low', 'adj_close', 'adj_volume']]

In [ ]:
spy_optiondata['date'] = pd.to_datetime(spy_optiondata['date'])
spy_optiondata['strike_price'] = spy_optiondata['strike_price'] / 1000.0

In [ ]:
spy_optiondata_enriched = pd.merge(spy_optiondata, spy_tickerdata_reduced, on='date', how='left')

In [ ]:
display(spy_optiondata_enriched)

In [ ]:
spy_optiondata_enriched.to_csv('combinedata.csv', index=False)

In [ ]:
spydata_cleaned = spy_optiondata_enriched.dropna(subset=['impl_volatility', 'delta', 'gamma', 'vega', 'theta'])
spydata_dropna = spydata_cleaned.reset_index(drop=True)
spydata_dropna.to_csv('combinedata_dropna.csv', index=False)

## Load useful data

In [3]:
filepath = 'combinedata.csv'
filepath1 = 'combinedata_dropna.csv'
options_df = pd.read_csv(filepath)

In [4]:
display(options_df)

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,adj_open,adj_high,adj_low,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,242.053393,242.930005,241.655754,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10448189,2023-02-28,2025-03-21,2022-12-28,P,600.0,200.50,205.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448190,2023-02-28,2025-03-21,NaN,P,605.0,205.50,210.50,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448191,2023-02-28,2025-03-21,NaN,P,610.0,210.00,215.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
10448192,2023-02-28,2025-03-21,NaN,P,615.0,215.00,220.00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,391.240559,393.259649,390.176843,390.285185,96141367.0
